# Solving Business Problems with AI

## Objective
Develop a proof-of-concept application to intelligently process email order requests and customer inquiries for a fashion store. The system should accurately categorize emails as either product inquiries or order requests and generate appropriate responses using the product catalog information and current stock status.

## Task Description

### Inputs

Google Spreadsheet **[Document](https://docs.google.com/spreadsheets/d/14fKHsblfqZfWj3iAaM2oA51TlYfQlFT4WKo52fVaQ9U)** containing:

- **Products**: List of products with fields including product ID, name, category, stock amount, detailed description, and season.

- **Emails**: Sequential list of emails with fields such as email ID, subject, and body.

### Instructions

- Implement all requirements using advanced Large Language Models (LLMs) to handle complex tasks, process extensive data, and generate accurate outputs effectively.
- Use Retrieval-Augmented Generation (RAG) and vector store techniques where applicable to retrieve relevant information and generate responses.
- You are provided with a temporary OpenAI API key granting access to GPT-4o, which has a token quota. Use it wisely or use your own key if preferred.
- Address the requirements in the order listed. Review them in advance to develop a general implementation plan before starting.
- Your deliverables should include:
   - Code developed within this notebook.
   - A single spreadsheet containing results, organized across separate sheets.
   - Comments detailing your thought process.
- You may use additional libraries (e.g., langchain) to streamline the solution. Use libraries appropriately to align with best practices for AI and LLM tools.
- Use the most suitable AI techniques for each task. Note that solving tasks with traditional programming methods will not earn points, as this assessment evaluates your knowledge of LLM tools and best practices.

### Requirements

#### 1. Classify emails
    
Classify each email as either a _**"product inquiry"**_ or an _**"order request"**_. Ensure that the classification accurately reflects the intent of the email.

**Output**: Populate the **email-classification** sheet with columns: email ID, category.

#### 2. Process order requests
1.   Process orders
  - For each order request, verify product availability in stock.
  - If the order can be fulfilled, create a new order line with the status “created”.
  - If the order cannot be fulfilled due to insufficient stock, create a line with the status “out of stock” and include the requested quantity.
  - Update stock levels after processing each order.
  - Record each product request from the email.
  - **Output**: Populate the **order-status** sheet with columns: email ID, product ID, quantity, status (**_"created"_**, **_"out of stock"_**).

2.   Generate responses
  - Create response emails based on the order processing results:
      - If the order is fully processed, inform the customer and provide product details.
      - If the order cannot be fulfilled or is only partially fulfilled, explain the situation, specify the out-of-stock items, and suggest alternatives or options (e.g., waiting for restock).
  - Ensure the email tone is professional and production-ready.
  - **Output**: Populate the **order-response** sheet with columns: email ID, response.

#### 3. Handle product inquiry

Customers may ask general open questions.
  - Respond to product inquiries using relevant information from the product catalog.
  - Ensure your solution scales to handle a full catalog of over 100,000 products without exceeding token limits. Avoid including the entire catalog in the prompt.
  - **Output**: Populate the **inquiry-response** sheet with columns: email ID, response.

## Evaluation Criteria
- **Advanced AI Techniques**: The system should use Retrieval-Augmented Generation (RAG) and vector store techniques to retrieve relevant information from data sources and use it to respond to customer inquiries.
- **Tone Adaptation**: The AI should adapt its tone appropriately based on the context of the customer's inquiry. Responses should be informative and enhance the customer experience.
- **Code Completeness**: All functionalities outlined in the requirements must be fully implemented and operational as described.
- **Code Quality and Clarity**: The code should be well-organized, with clear logic and a structured approach. It should be easy to understand and maintain.
- **Presence of Expected Outputs**: All specified outputs must be correctly generated and saved in the appropriate sheets of the output spreadsheet. Ensure the format of each output matches the requirements—do not add extra columns or sheets.
- **Accuracy of Outputs**: The accuracy of the generated outputs is crucial and will significantly impact the evaluation of your submission.

We look forward to seeing your solution and your approach to solving real-world problems with AI technologies.

# Prerequisites

### Configure OpenAI API Key.

In [1]:
# Install the OpenAI Python package.
%pip install openai
!pip install langchain
!pip install -U langchain-community
!pip install faiss-cpu
!pip install tiktoken
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.6/375.6 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.2 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

**IMPORTANT: If you are going to use our custom API Key then make sure that you also use custom base URL as in example below. Otherwise it will not work.**

In [2]:
# Code example of OpenAI communication

import openai
from openai import OpenAI
from langchain.embeddings.openai import OpenAIEmbeddings

openai_api_key ='a0BIj000001cSt3MAE'
openai.api_key = openai_api_key
client = OpenAI(
    # In order to use provided API key, make sure that models you create point to this custom base URL.
    base_url='https://47v4us7kyypinfb5lcligtc3x40ygqbs.lambda-url.us-east-1.on.aws/v1/',
    # The temporary API key giving access to ChatGPT 4o model. Quotas apply: you have 500'000 input and 500'000 output tokens, use them wisely ;)
    api_key=openai_api_key
)

completion = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "user", "content": "Hello!"}
  ]
)

print(completion.choices[0].message)

embeddings_model = OpenAIEmbeddings(openai_api_key=openai_api_key)

ChatCompletionMessage(content='Hi there! How can I assist you today?', refusal=None, role='assistant', function_call=None, tool_calls=None)


<ipython-input-2-0f3ba5a6d7cd>:25: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings_model = OpenAIEmbeddings(openai_api_key=openai_api_key)


In [3]:
try:
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": "Hello!"}]
    )
    print("API key is working:", completion.choices[0].message)
except Exception as e:
    print("Error:", e)

API key is working: ChatCompletionMessage(content='Hi there! How can I assist you today?', refusal=None, role='assistant', function_call=None, tool_calls=None)


In [4]:
# Code example of reading input data

import pandas as pd
from IPython.display import display

def read_data_frame(document_id, sheet_name):
    export_link = f"https://docs.google.com/spreadsheets/d/{document_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
    return  pd.read_csv(export_link)

document_id = '14fKHsblfqZfWj3iAaM2oA51TlYfQlFT4WKo52fVaQ9U'
products_df = read_data_frame(document_id, 'products')
emails_df = read_data_frame(document_id, 'emails')

# Display first 3 rows of each DataFrame
display(products_df.head(3))
display(emails_df.head(3))

,product_id,name,category,description,stock,seasons,price
0,RSG8901,Retro Sunglasses,Accessories,Transport yourself back in time with our retro...,1,"Spring, Summer",26.99
1,SWL2345,Sleek Wallet,Accessories,Keep your essentials organized and secure with...,5,All seasons,30.00
2,VSC6789,Versatile Scarf,Accessories,Add a touch of versatility to your wardrobe wi...,6,"Spring, Fall",23.00


,email_id,subject,message
0,E001,Leather Wallets,"Hi there, I want to order all the remaining LT..."
1,E002,Buy Vibrant Tote with noise,"Good morning, I'm looking to buy the VBT2345 V..."
2,E003,Need your help,"Hello, I need a new bag to carry my laptop and..."


# Task 1. Classify emails

In [5]:
from google.colab import files
uploaded = files.upload()

Saving ai-test-436218-bb896f943c5b.json to ai-test-436218-bb896f943c5b.json


In [6]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Define the scope for accessing Google Sheets
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']

# Load credentials from the JSON keyfile you downloaded
# Ensure the path is correct and the file exists
creds = ServiceAccountCredentials.from_json_keyfile_name('ai-test-436218-bb896f943c5b.json', scope)

# Authenticate the client
goog = gspread.authorize(creds)

# Open the Google Sheet by name
spreadsheet = goog.open('Copy of Solving Business Problems with AI')


In [7]:
try:
    classification_sheet = spreadsheet.worksheet('email-classification')
except gspread.exceptions.WorksheetNotFound:
    classification_sheet = spreadsheet.add_worksheet(title='email-classification', rows="1000", cols="2")

# Add headers (if it's a new sheet)
classification_sheet.clear()  # Clear the sheet if not empty
classification_sheet.append_row(["email ID", "category"])

{'spreadsheetId': '16Jsk9IaH5AKo9zDm_EV9GBIBeBYMYTM5m0kPIHyr1jk',
 'updates': {'spreadsheetId': '16Jsk9IaH5AKo9zDm_EV9GBIBeBYMYTM5m0kPIHyr1jk',
  'updatedRange': "'email-classification'!A1:B1",
  'updatedRows': 1,
  'updatedColumns': 2,
  'updatedCells': 2}}

In [8]:
def classify_email_gpt(subject, message):
    prompt = f"""
    You are an AI that classifies emails. Here is an email:
    Subject: {subject}
    Body: {message}

    Classify this email as either an "order request" if the email is wants to put an order in for an item or "product inquiry" if anything else. Reply with only the classification.
    """

    # Sending a request to the GPT-4o API
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )


    # Get the classification result from the response
    # Use dot notation (.) to access the content attribute
    classification = response.choices[0].message.content.strip()
    return classification

# Example: Classify the first few emails from the dataset
for index, email in emails_df.iterrows():
    subject = email['subject']
    message = email['message']
    email_id = email['email_id']

    # Classify the email
    category = classify_email_gpt(subject, message)

    # Print classification for review
    print(f"Email ID: {email_id}, Category: {category}")

    classification_sheet.append_row([email_id, category])

Email ID: E001, Category: order request
Email ID: E002, Category: order request
Email ID: E003, Category: product inquiry
Email ID: E004, Category: order request
Email ID: E005, Category: product inquiry
Email ID: E006, Category: product inquiry
Email ID: E007, Category: Order request
Email ID: E008, Category: order request
Email ID: E009, Category: product inquiry
Email ID: E010, Category: order request
Email ID: E011, Category: product inquiry
Email ID: E012, Category: Product inquiry
Email ID: E013, Category: product inquiry
Email ID: E014, Category: order request
Email ID: E015, Category: product inquiry
Email ID: E016, Category: product inquiry
Email ID: E017, Category: order request
Email ID: E018, Category: order request
Email ID: E019, Category: order request
Email ID: E020, Category: product inquiry
Email ID: E021, Category: product inquiry


# Task 2. Process order requests

In [9]:


# Initialize embeddings model with the correct API key
embeddings_model = OpenAIEmbeddings(model="text-embedding-ada-002", openai_api_key=('sk-a0BIj000001cSt3MAE'))



In [10]:
# Prepare product data for vectorization
product_texts = products_df.apply(
    lambda row: f"Product ID: {row['product_id']}, Name: {row['name']}, Category: {row['category']}, Description: {row['description']}", axis=1).to_list()



In [11]:
try:
    product_vectors = client.embeddings.create(
        model="text-embedding-ada-002",  # Replace with your desired model
        input=product_texts
    )
    print("Embeddings computed successfully.")
except Exception as e:
    print("Error:", e)

Embeddings computed successfully.


In [12]:
# Extract embeddings from the response object
product_embeddings = [embedding.embedding for embedding in product_vectors.data]

# Convert the list of embeddings into a numpy array
product_vectors_np = np.array(product_embeddings).astype('float32')

print(f"Converted {len(product_embeddings)} product embeddings to a numpy array.")

Converted 99 product embeddings to a numpy array.


In [13]:
import faiss

# Get the dimension of the embeddings (length of each vector)
dimension = product_vectors_np.shape[1]  # Assuming embeddings are 2D (num_products, embedding_dimension)

# Create a FAISS index using the L2 (Euclidean) distance metric
index_faiss = faiss.IndexFlatL2(dimension) # Changed 'index' to 'index_faiss'

# Step 2: Add embeddings to the FAISS index
index_faiss.add(product_vectors_np) # Changed 'index' to 'index_faiss'

print(f"Added {index_faiss.ntotal} products to the FAISS index.") # Changed 'index' to 'index_faiss'

Added 99 products to the FAISS index.


In [14]:
def retrieve_similar_products(query_text, k=5):
    # Check if query_text is a string
    if isinstance(query_text, str):
        # Step 1: Convert the query text into an embedding using the same model
        query_embedding = client.embeddings.create(
            model="text-embedding-ada-002",
            input=[query_text]
        )

        # Extract the embedding from the response
        query_vector = np.array([query_embedding.data[0].embedding]).astype('float32')
    else:
        # If query_text is not a string (presumably an embedding), assign it directly to query_vector
        query_vector = np.array([query_text]).astype('float32') # Convert query_text to a NumPy array and ensure it's 2D

    # Step 2: Perform FAISS search (find the k nearest neighbors)
    distances, indices = index_faiss.search(query_vector, k)

    # Step 3: Retrieve the product IDs and distances
    similar_products = products_df.iloc[indices[0]]['product_id'].tolist()
    return similar_products, distances[0].tolist()

# Example query
query = "Looking for a product similar to Product 1 in Category 1"
similar_products, distances = retrieve_similar_products(query, k=3)

print(f"Similar products to '{query}': {similar_products} with distances {distances}")

Similar products to 'Looking for a product similar to Product 1 in Category 1': ['TLR5432', 'SKR3210', 'CPL0123'] with distances [0.46233394742012024, 0.466063916683197, 0.46759676933288574]


In [15]:
# Load the classification sheet into a DataFrame
classification_df = pd.DataFrame(classification_sheet.get_all_records())


In [16]:
order_request_emails = classification_df[classification_df['category'] == 'order request']


In [17]:
order_request_emails

,email ID,category
0,E001,order request
1,E002,order request
3,E004,order request
7,E008,order request
9,E010,order request
13,E014,order request
16,E017,order request
17,E018,order request
18,E019,order request


In [18]:
def check_stock_ai(message, client, products_df):
  # Use OpenAI to extract order information
  prompt = f"""
  You are an AI assistant helping to process orders.
  Analyze the following message and extract the product IDs and quantities.
  Product IDs are in the format of three capital letters and then four numbers.
  Some messages may include a space between the letters and numbers of the Product ID, but delete those spaces.
  If a customer requests "all" units, consider it as requesting the entire stock.
  If the message doesn't include the product ID, please include the product name instead.

  Message: {message}

  Respond in the following JSON format:
  {{
    "products": [
      {{
        "product_id": "product_id",
        "product_name": "product_name",
        "quantity": integer
      }},
      {{
        "product_id": "product_id",
        "product_name": "product_name",
        "quantity": integer
      }}
    ]
  }}
  """

  response = client.chat.completions.create(
      model="gpt-4",
      messages=[{"role": "user", "content": prompt}]
  )

  order_info = response.choices[0].message.content.strip()

  import json

  try:
    order_info = json.loads(order_info)
  except json.JSONDecodeError:
      print(f"Invalid JSON: {order_info}")
      return "Could not process order."

  stock_info_list = []
  for product in order_info['products']:
    product_id = product.get('product_id')
    product_name = product.get('product_name')

    if product_id:
      # Check if the product exists in products_df using product_id
      product_stock = products_df[products_df['product_id'] == product_id]['stock']
      if len(product_stock) == 0:
          stock_info_list.append(f"Product {product_id} not found.")
          continue
    elif product_name:
      # Check if the product exists in products_df using product_name
      product_stock = products_df[products_df['name'] == product_name]['stock']
      if len(product_stock) == 0:
          stock_info_list.append(f"Product {product_name} not found.")
          continue
      product_id = products_df[products_df['name'] == product_name]['product_id'].values[0] # Get product_id
    else:
      stock_info_list.append(f"Could not find product ID or name.")
      continue

    stock = product_stock.values[0]

    if 'quantity' in product:
      requested_quantity = product['quantity']
      if stock is None:
        stock_info_list.append(f"Product {product_id} stock information is not available.")
      elif requested_quantity == "all":
        stock_info_list.append(f"Product {product_id} is in stock (requested all, available {stock}).")
      elif stock >= requested_quantity:
        stock_info_list.append(f"Product {product_id} is in stock (requested {requested_quantity}, available {stock}).")
      else:
        stock_info_list.append(f"Product {product_id} is out of stock (requested {requested_quantity}, available {stock}).")
    else:
      stock_info_list.append(f"Could not extract quantity from order for product {product_id}.")

  return "\n".join(stock_info_list)

In [19]:
email_ids = order_request_emails['email ID'].tolist()
# Loop through the filtered emails and check stock
for email_id in email_ids:
  message = emails_df[emails_df['email_id'] == email_id]['message'].values[0]
  stock_info = check_stock_ai(message, client, products_df)
  print(f"Email {email_id}: {stock_info}")

Email E001: Product LTH0976 is in stock (requested all, available 4).
Email E002: Product VBT2345 is in stock (requested 1, available 4).
Email E004: Product SFT1098 is in stock (requested 3, available 8).
Product SFT1098 is in stock (requested 4, available 8).
Email E008: Product null not found.
Email E010: Product RSG8901 is in stock (requested 1, available 1).
Email E014: Product SWL2345 is in stock (requested 1, available 5).
Invalid JSON: The message doesn't provide any specific product IDs or names, and no specific quantities have been given. Without this crucial information, an appropriate extraction can't be executed. Additional information from the customer is needed to proceed with the order.
Email E017: Could not process order.
Email E018: Product RSG8901 is out of stock (requested 2, available 1).
Email E019: Product CBT8901 is in stock (requested all, available 2).
Product FZZ1098 is in stock (requested all, available 2).
Product Retro sunglasses not found.


In [20]:
try:
    order_status_worksheet = spreadsheet.worksheet("order-status")
except gspread.exceptions.WorksheetNotFound:
    order_status_worksheet = spreadsheet.add_worksheet("order-status", rows=100, cols=10)
    # Add header row
    order_status_worksheet.append_row(["email ID", "product ID", "status", "quantity"])

In [21]:
def update_order_status(email_id, stock_info, order_status_worksheet, products_df):
    # Split stock information into individual product lines
    stock_info_lines = stock_info.split('\n')

    for line in stock_info_lines:
        if "in stock" in line:
            # Extract product ID and available stock
            product_id = line.split(" ")[1]
            available_stock = int(line.split("available ")[-1].replace(").", ""))

            # Update order status worksheet
            order_status_worksheet.append_row([email_id, product_id, "created", 1]) # Add quantity 1 for in stock items

            # Update stock level
            products_df.loc[products_df['product_id'] == product_id, 'stock'] = available_stock - 1

        elif "out of stock" in line:
            # Extract product ID and requested quantity
            product_id = line.split(" ")[1]
            requested_quantity = int(line.split("requested ")[-1].split(",")[0])

            # Update order status worksheet
            order_status_worksheet.append_row([email_id, product_id, "out of stock", requested_quantity])


order_request_emails = classification_df[classification_df['category'] == 'order request']
email_ids = order_request_emails['email ID'].tolist()

# Loop through the filtered emails and check stock
for email_id in email_ids:
    message = emails_df[emails_df['email_id'] == email_id]['message'].values[0]
    stock_info = check_stock_ai(message, client, products_df)
    print(f"Email {email_id}: {stock_info}")

    # Update order status and stock levels
    update_order_status(email_id, stock_info, order_status_worksheet, products_df)

# Update the products worksheet with the new stock levels
spreadsheet.worksheet("products").update([products_df.columns.values.tolist()] + products_df.values.tolist())

Email E001: Product LTH0976 is in stock (requested all, available 4).
Email E002: Product VBT2345 is in stock (requested 1, available 4).
Email E004: Product SFT1098 is in stock (requested 3, available 8).
Product SFT1098 is in stock (requested 4, available 8).
Email E008: Product None not found.
Email E010: Product RSG8901 is in stock (requested 1, available 1).
Email E014: Product SWL2345 is in stock (requested 1, available 5).
Invalid JSON: From the provided message, I couldn't identify any specific product IDs, quantities, or even product names. It seems the customers is referring to a popular item but without providing more specific details. Thus, it is impossible to correctly fill the needed JSON format. 

Here is my suggested response:

{
  "products": [
    {
      "product_id": "",
      "product_name": "Popular Item",
      "quantity": null
    }
  ]
}  

We may need to reach out to the customer for more specific information to complete the order.
Email E017: Could not proces

{'spreadsheetId': '16Jsk9IaH5AKo9zDm_EV9GBIBeBYMYTM5m0kPIHyr1jk',
 'updatedRange': 'products!A1:G100',
 'updatedRows': 100,
 'updatedColumns': 7,
 'updatedCells': 700}

In [22]:
# Get the "order-response" worksheet or create it if it doesn't exist
try:
    order_response_worksheet = spreadsheet.worksheet("order-response")
except gspread.exceptions.WorksheetNotFound:
    order_response_worksheet = spreadsheet.add_worksheet("order-response", rows=100, cols=10)
    # Add header row
    order_response_worksheet.append_row(["email ID", "response"])

def generate_response(email_id, order_status_df, products_df):
    """Generates a response email based on order status."""

    order_items = order_status_df[order_status_df['email ID'] == email_id]

    created_items = order_items[order_items['status'] == 'created']
    out_of_stock_items = order_items[order_items['status'] == 'out of stock']

    response = f"Dear Customer,\n\nThank you for your recent order! Here's an update:\n\n"

    if not created_items.empty:
        response += "The following items have been confirmed and are being processed:\n"
        for _, row in created_items.iterrows():
            product_id = row['product ID']
            product_name = products_df[products_df['product_id'] == product_id]['name'].values[0]
            response += f"- {product_name} (Product ID: {product_id})\n"
        response += "\n"

    if not out_of_stock_items.empty:
        response += "Unfortunately, the following items are currently out of stock:\n"
        for _, row in out_of_stock_items.iterrows():
            product_id = row['product ID']
            product_name = products_df[products_df['product_id'] == product_id]['name'].values[0]
            response += f"- {product_name} (Product ID: {product_id})\n"
        response += "\nWe apologize for any inconvenience this may cause.\n\n"

    if not created_items.empty:
        response += "You will receive a separate email with shipping information once your order is dispatched.\n\n"

    response += "Sincerely,\nThe Customer Support Team"

    return response

order_request_emails = classification_df[classification_df['category'] == 'order request']
email_ids = order_request_emails['email ID'].tolist()

# Loop through the filtered emails and check stock
for email_id in email_ids:
    message = emails_df[emails_df['email_id'] == email_id]['message'].values[0]
    stock_info = check_stock_ai(message, client, products_df)

    # Generate response email
    order_status_df = pd.DataFrame(order_status_worksheet.get_all_records())
    response = generate_response(email_id, order_status_df, products_df)

    # Append response to order-response worksheet
    order_response_worksheet.append_row([email_id, response])

Invalid JSON: The given message doesn't include any product IDs nor does it provide a specific product name or quantity. Due to these missing details, it isn't possible to accurately fill in a JSON format response without making assumptions. The customer only mentioned a "popular item" which is not enough data to extract valid information.

With further specific information, the assistant can provide an accurate JSON format response.


# Task 3. Handle product inquiry

In [23]:
try:
    inquiry_response_worksheet = spreadsheet.worksheet("inquiry-response")
except gspread.exceptions.WorksheetNotFound:
    inquiry_response_worksheet = spreadsheet.add_worksheet("inquiry-response", rows=100, cols=10)
    inquiry_response_worksheet.append_row(["email ID", "response"])


In [26]:
def generate_inquiry_response(email_id, message, products_df):
    """Generates a response to a product inquiry email."""

    # Use OpenAI to generate a response
    prompt = f"""
    You are a customer service AI assistant.
    Respond to the following customer email inquiry in a professional and helpful tone.
    Use information from the provided product catalog to answer their questions.
    If you cannot find an answer to their question, politely state that you are unable to find information about that.

    Customer email: {message}

    Product Catalog: {products_df.to_json()}
    """
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",  # You can use gpt-4 if you have access
        messages=[{"role": "user", "content": prompt}]
    )

    response_text = response.choices[0].message.content.strip()
    return response_text

In [27]:
# Get product inquiries
product_inquiry_emails = classification_df[classification_df['category'] == 'product inquiry']
email_ids = product_inquiry_emails['email ID'].tolist()

# Loop through the filtered emails and generate responses
for email_id in email_ids:
    message = emails_df[emails_df['email_id'] == email_id]['message'].values[0]
    response = generate_inquiry_response(email_id, message, products_df)
    inquiry_response_worksheet.append_row([email_id, response])